# Tic-Tac-Toe: Using Minimax and Alpha-Beta Pruning

### Necessary Imports

In [1]:
import copy
import math
import random

### Base Variables

In [2]:
X = "X"
O = "O"
EMPTY = None

### Some Fundamental Functionlity

In [3]:
# Displays the board in 2D
def display(board):
    """Display the board."""
    print("-------------")
    for row in board:
        print("|", end="")
        for cell in row:
            if cell is None:
                print(" - ", end="|")
            else:
                print('',cell,'', end="|")
        print()
        print("-------------")

In [4]:
# Returns starting state of the board.
def initial_state():
    return [[EMPTY, EMPTY, EMPTY],[EMPTY, EMPTY, EMPTY],[EMPTY, EMPTY, EMPTY]]

In [5]:
# Returns player 'X' or 'O' who has the next turn on a board.
def player(board):
    count = sum([1 for row in board for cell in row if cell])
    return 'O' if count % 2 else 'X'

In [6]:
# Returns set of all possible actions (i, j) available on the board.
def actions(board):
    res = set()
    for i, r in enumerate(board):
        res.update((i, j) for j, c in enumerate(r) if c == EMPTY)
    return res

In [7]:
# Returns the board that results from making move (i, j) on the board.
def result(board, action):
    curr_player = player(board)
    new_board = [row[:] for row in board]
    new_board[action[0]][action[1]] = curr_player
    return new_board

In [8]:
# Returns the winner of the game, if a player is present at three consecutive horizontal boxes.
def get_horizontal_winner(board):
    for row in board:
        if all(cell == row[0] and cell is not None for cell in row):
            return row[0]
    return None

In [9]:
# Returns the winner of the game, if a player is present at three consecutive vertical boxes.
def get_vertical_winner(board):
    for col in range(len(board[0])):
        if all(board[row][col] == board[0][col] for row in range(len(board))):
            return board[0][col]
    return None

In [10]:
# Returns the winner of the game, if a player is present at three consecutive diagonal boxes.
def get_diagonal_winner(board):
    # Check main diagonals
    if all(board[i][i] == board[0][0] for i in range(len(board))):
        return board[0][0]
    # Check flipped diagonals
    if all(board[i][len(board)-1-i] == board[0][len(board)-1] for i in range(len(board))):
        return board[0][len(board)-1]
    return None

In [11]:
# Returns the winner of the game, if any.
def winner(board):
    return get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None

In [12]:
# Returns True if game is over, False otherwise.
def terminal(board):
    if winner(board):
        return True
    if any(EMPTY in row for row in board):
        return False
    return True

In [13]:
# Returns 1 if 'X' has won, -1 if 'O' has won, 0 otherwise.
def utility(board):
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

# MinMax Algorithm

- The `max_val` function takes a `board` as input and returns the maximum possible value that the current player can achieve from the given board state. It recursively evaluates all the possible moves that can be made by the current player and returns the maximum value from the minimum values returned by the opponent.

- If the given `board` is already in a terminal state (i.e., the game is over), the `utility` function is called to determine the score of the terminal state.

- The function then initializes a variable `v` to negative infinity, which will be used to store the maximum value found so far. It then iterates over all the possible actions that can be taken from the current `board` state, and for each action, it calls the min_val function on the resulting board state. The `min_val` function will return the minimum value that the opponent can achieve from the resulting state. The `max` function is then used to update the `   ` variable to the maximum value between the current `v` and the minimum value returned by `min_val`. Finally, the function returns the maximum value found.

In [14]:
def max_val(board):
    if terminal(board):
        return utility(board)
    max_sol = float('-inf')
    for act in actions(board):
        max_sol = max(max_sol , min_val(result(board,act)))
    return max_sol

- The `min_val` function is similar to the `max_val` function, but instead of returning the maximum value that the current player can achieve, it returns the minimum value that the opponent can achieve. It does this by recursively evaluating all the possible moves that the opponent can make and returns the minimum value from the maximum values returned by the current player.

- If the given `board` is already in a terminal state (i.e., the game is over), the `utility` function is called to determine the score of the terminal state.

- The function initializes a variable `v` to positive infinity, which will be used to store the minimum value found so far. It then iterates over all the possible actions that can be taken from the current `board` state, and for each action, it calls the `max_val` function on the resulting board state. The `max_val` function will return the maximum value that the current player can achieve from the resulting state. The `min` function is then used to update the `v` variable to the minimum value between the current `v` and the maximum value returned by `max_val`. Finally, the function returns the minimum value found.


In [15]:
def min_val(board):
    if terminal(board):
        return utility(board)
    max_sol = float('inf')
    for act in actions(board):
        max_sol = min(max_sol , max_val(result(board,act)))
    return max_sol

- An implementation of the minimax algorithm for the game of Tic Tac Toe. The algorithm is used to determine the best move for the current player in the game. The function `minimax(board)` takes a `board` as input and returns the best move for the current player.

- The function first checks if the `board` is the initial state of the game. If it is, then it returns a random move. Otherwise, it determines the current player and initializes the `action_to_return` variable to `None`.

- If the current player is `X`, then the function initializes the `val` variable to negative infinity and iterates over all the possible actions that can be taken on the `board`. For each action, it calls the `min_val` function on the result of applying the action to the `board`. If the resulting value is greater than `val`, then `val` is updated to the resulting value and the `action_to_return` variable is set to the current action.

- If the current player is `O`, then the function initializes the `val` variable to positive infinity and iterates over all the possible actions that can be taken on the `board`. For each action, it calls the `max_val` function on the result of applying the action to the `board`. If the resulting value is less than `val`, then `val` is updated to the resulting value and the `action_to_return` variable is set to the current action.

- Finally, the function returns the `action_to_return` variable, which is the best move for the current player.

In [16]:
def minimax(board):
    # Check if the board is the initial state
    if board == [[EMPTY, EMPTY, EMPTY],[EMPTY, EMPTY, EMPTY],[EMPTY, EMPTY, EMPTY]]:
        # Return a random move
        return random.sample(actions(board), 1)[0]
    # Determine the current player
    current_player = player(board)
    # Initialize the action_to_return variable to None
    action_to_return = None
    # If the current player is X
    if (current_player == 'X'):
        # Initialize the val variable to negative infinity
        val = float('-inf')
        # Iterate over all the possible actions that can be taken on the board
        for act in actions(board):
            # Call the min_val function on the result of applying the action to the board
            v1 = min_val(result(board,act))
            # If the resulting value is greater than val, update val and action_to_return
            if(v1 > val):
                val = v1
                action_to_return = act
    # If the current player is O
    elif (current_player == 'O'):
        # Initialize the val variable to positive infinity
        val = float('inf')
        # Iterate over all the possible actions that can be taken on the board
        for act in actions(board):
        # Call the max_val function on the result of applying the action to the board
            v1 = max_val(result(board,act))
        # If the resulting value is less than val, update val and action_to_return
            if(v1<val):
                val = v1
                action_to_return = act
    # Return the best move for the current player
    return action_to_return

# Running Script

- Code implementation of the game of Tic Tac Toe. The code starts by initializing the `user` variable to `None` and the `board` variable to the initial state of the game. It then sets the `ai_turn` variable to `False` and prompts the user to choose a player. 

- The code then enters a loop that continues until the game is over. In each iteration of the loop, it checks if the game is over by calling the `terminal` function on the `board`. If the game is over, it prints the result of the game and breaks out of the loop. Otherwise, it determines the current player by calling the `player` function on the `board` and prints the current player and the user.

- If the game is not over and the current player is not the user, then the code sets the `ai_turn` variable to `True` and selects a move for the AI player by calling the `minimax` function on the `board`. It then applies the selected move to the `board` by calling the `result` function on the `board` and the selected move. Finally, it sets the `ai_turn` variable to `False` and prints the updated `board`.

- If the game is not over and the current player is the user, then the code sets the `ai_turn` variable to `True` and prompts the user to enter the position to move. It then applies the selected move to the `board` by calling the `result` function on the `board` and the selected move. Finally, it prints the updated `board`.

In [17]:
if __name__ == "__main__":
    board = initial_state()
    ai_turn = False
    plyer = 'X'
    print("You are X")
    while(not terminal(board)):
        if ai_turn == True:
            aimove = minimax(board)
            board = result(board,aimove)
            ai_turn = False
            print("AI Turn! Move Chosen: " + str(aimove))
        else:
            ai_turn = True
            valid_move = 0
            print("Choose from Following Moves")
            print(actions(board))
            while valid_move == 0:
                x = input("Enter Board Row:  ")
                y = input("Enter Board Column:  ")
                uact = (int(x),int(y))
                print("You Choosed: " ,end=' ')
                print(uact)
                if(uact in actions(board)):
                    board = result(board,uact)
                    valid_move = 1
                    break
        print('-'*100)
        display(board)
        
    winner = utility(board)
    if winner == 0:
        print("Game Draw!")
    
    elif plyer == 'X':
        if winner == 1:
            print("You Won!")
        elif winner == -1:
            print("You Lost!")
    elif plyer == 'O':
        if winner == -1:
            print("You Won!")
        elif winner== 1:
            print("You Lost!")


You are X
Choose from Following Moves
{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
Enter Board Row:  0
Enter Board Column:  0
You Choosed:  (0, 0)
----------------------------------------------------------------------------------------------------
-------------
| X | - | - |
-------------
| - | - | - |
-------------
| - | - | - |
-------------
AI Turn! Move Chosen: (1, 1)
----------------------------------------------------------------------------------------------------
-------------
| X | - | - |
-------------
| - | O | - |
-------------
| - | - | - |
-------------
Choose from Following Moves
{(0, 1), (1, 2), (2, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
Enter Board Row:  0
Enter Board Column:  1
You Choosed:  (0, 1)
----------------------------------------------------------------------------------------------------
-------------
| X | X | - |
-------------
| - | O | - |
-------------
| - | - | - |
-------------
AI Turn! Move Chosen: (0, 2)
---------------------